In [1]:
# import libraries

import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier

In [5]:
# install the hyperopt module if using the tensorflow docker container
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org hyperopt

In [6]:
# import hyper parameter tuning libraries

# hp: define the hyperparameter space
# fmin: optimization function
# Trials: to evaluate the different searched hyperparameters
from hyperopt import hp, fmin

# the search algoriths
from hyperopt import tpe

In [7]:
# load dataset
breast_cancer_x, breast_cancer_y = load_breast_cancer(return_X_y = True)
X = pd.DataFrame(breast_cancer_x)
y = pd.Series(breast_cancer_y).map({0:1,1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
# the target
# percentage of benign (0) and malignant (1) tumours
y.value_counts()/len(y)

0    0.627417
1    0.372583
Name: count, dtype: float64

In [9]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

In [11]:
# determine teh hyperparameter space
param_grid= {
    'hidden_layer_sizes':hp.choice('hidden_layer_sizes',
                                   [(10),(10,20),(10,20,30)]),
    'activation':hp.choice('activation',['tanh','relu']),
    'solver':hp.choice('solver',['adam','sgd']),
    'learning_rate_init':hp.loguniform('learning_rate_init',
                                        np.log(0.001), np.log(1)),
}

In [12]:
# the objective function takes the hyperparameter space as input
def objective(params):
    # dictionary to indicate which value from the space
    # to attribute to each value of the hyperparameter in xgb
    params_dict = {
        'hidden_layer_sizes': params['hidden_layer_sizes'],
        'activation': params['activation'],
        'solver': params['solver'],
        'learning_rate_init': params['learning_rate_init'],
        'random_state': 1000,
        'max_iter': 100,
    }

    # with ** we pass teh items to the dictionary as parameters
    # to the xgb
    mlp = MLPClassifier(**params_dict)

    # train with cv
    score = cross_val_score(mlp, X_train, y_train,
                            scoring='accuracy', cv=3,
                            n_jobs=4).mean()

    # to minimize, we negate the score
    return -score

In [14]:
# fmin performs the minimization
# rand.suggest samples the parameters at random
# i.e., performs random search
search = fmin(
    fn = objective,
    space = param_grid,
    max_evals = 50,
    rstate = np.random.default_rng(42),
    algo = tpe.suggest # randomized search
)    

 16%|███████▊                                         | 8/50 [00:01<00:05,  7.13trial/s, best loss: -0.8793005240373661]

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


 30%|██████████████▍                                 | 15/50 [00:02<00:02, 14.49trial/s, best loss: -0.8793005240373661]

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


 42%|████████████████████▏                           | 21/50 [00:02<00:01, 16.67trial/s, best loss: -0.9221158958001063]

...

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


 98%|███████████████████████████████████████████████ | 49/50 [00:04<00:00, 16.01trial/s, best loss: -0.9246221614642667]

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.28trial/s, best loss: -0.9246221614642667]


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
search

{'activation': np.int64(1),
 'hidden_layer_sizes': np.int64(1),
 'learning_rate_init': np.float64(0.001049742701720731),
 'solver': np.int64(0)}